Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД.
Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты). Для тех, кто выполнил задание с Росконтролем - напишите запрос для поиска продуктов с рейтингом не ниже введенного или качеством не ниже введенного (то есть цифра вводится одна, а запрос проверяет оба поля)
Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта.

In [1]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
#import pandas as pd
from pymongo import MongoClient

In [2]:
client = MongoClient('127.0.0.1', 27017)
db = client['test_v1_01']
vacancy_mongo = db.vacancy

In [3]:
headers = {'User-agent': 'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2224.3 Safari/537.36'}

In [4]:
url = 'https://hh.ru/search/vacancy'

In [5]:
params = {'items_on_page': '20', 
          'area': '1',
          'page' : '0', 
          'clusters': 'true',
          'enable_snippets': 'true',
          'ored_clusters': 'true',
          'search_period': '7',
          'text': 'Python'}

In [6]:
response = requests.get(url, params=params, headers=headers).text

In [7]:
dom = bs(response,'html.parser')

In [8]:
#Определяем количество страниц с вакансиями

pages = dom.find_all('a',{'data-qa':'pager-page'})
no = []
for pages_no in pages:
    no.append(int(pages_no.getText()))
max_no = max(no)
print('Всего страниц с вакансиями    ', max_no)

Всего страниц с вакансиями     100


In [9]:
vacancy_serp = dom.find_all('div', {'class': 'vacancy-serp-item'})

In [10]:
vacancy_total = []

In [11]:
for page_no in range(0, max_no):
    for vacancy in vacancy_serp:
        
        vacancy_base = {}
    
        name_vacancy = vacancy.find('a',{'data-qa':'vacancy-serp__vacancy-title'}).getText()
        url_vakancy = vacancy.find('a',{'data-qa':'vacancy-serp__vacancy-title'}).get('href').split(sep='?')[0]


        try:
            money_vacancy = vacancy.find('span',{'data-qa':'vacancy-serp__vacancy-compensation'}).getText().replace(u'\u202f', u'')
        except:
            money_vacancy = None


        if money_vacancy == None:
            money_min  = None
            money_max  = None
            money_currency  = None
        else:
            mv = money_vacancy.split()
            lenth = len(mv)
            if lenth == 3:
                if mv[0] == 'до':
                    money_min  = None
                    money_max  = int(mv[1])
                    money_currency  = mv[2]
                else:
                    money_min  = int(mv[1])
                    money_max  = None
                    money_currency  = mv[2]
            else:
                money_min  = int(mv[0])
                money_max  = int(mv[2])
                money_currency  = mv[3]


        vacancy_base['name'] = name_vacancy
        vacancy_base['link'] = url_vakancy
        vacancy_base['min'] = money_min
        vacancy_base['max'] = money_max
        vacancy_base['currency'] = money_currency
        #vacancy_total.append(vacancy_base)
        
        vacancy_mongo.update_one({'link': vacancy_base['link']}, {'$set': vacancy_base}, upsert=True)
    
    print('Обработана страница    ', page_no)
    page_no += 1
    params['page'] = str(page_no)
    
    response = requests.get(url, params=params, headers=headers).text
    dom = bs(response,'html.parser')
    vacancy_serp = dom.find_all('div', {'class': 'vacancy-serp-item'})
    


Обработана страница     0
Обработана страница     1
Обработана страница     2
Обработана страница     3
Обработана страница     4
Обработана страница     5
Обработана страница     6
Обработана страница     7
Обработана страница     8
Обработана страница     9
Обработана страница     10
Обработана страница     11
Обработана страница     12
Обработана страница     13
Обработана страница     14
Обработана страница     15
Обработана страница     16
Обработана страница     17
Обработана страница     18
Обработана страница     19
Обработана страница     20
Обработана страница     21
Обработана страница     22
Обработана страница     23
Обработана страница     24
Обработана страница     25
Обработана страница     26
Обработана страница     27
Обработана страница     28
Обработана страница     29
Обработана страница     30
Обработана страница     31
Обработана страница     32
Обработана страница     33
Обработана страница     34
Обработана страница     35
Обработана страница     36
Обработана 

ConnectionError: HTTPSConnectionPool(host='hh.ru', port=443): Max retries exceeded with url: /search/vacancy?items_on_page=20&area=1&page=75&clusters=true&enable_snippets=true&ored_clusters=true&search_period=7&text=Python (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C66A79C550>: Failed to establish a new connection: [WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный отклик, или было разорвано уже установленное соединение из-за неверного отклика уже подключенного компьютера'))

In [ ]:
value = 125000

for result in vacancy_mongo.find(  {'$and': [{'currency': 'руб.'}, 
                                {'$or': [{'min': {'$gt': value}}, {'max': {'$gt': value}}]}]}):
    pprint(result)
